In [26]:
from keras.models import Sequential
from keras.layers import Dense, GRU
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import tensorflow.keras as keras

In [71]:
# Load the data sets and combine them into a single usable dataframe called data series
data_series = pd.read_csv('./data/data_avg_temp.csv')

# rename columns to be more descriptive
data_series.columns = ['date', 'avg_temp', 'avg_temp_anomaly']
data_series.drop(columns=['avg_temp_anomaly'], inplace=True)

# append to the dataframe
data_cool_degree = pd.read_csv('./data/data_cool_degree_days.csv')
data_heat_degree = pd.read_csv('./data/data_heat_degree_days.csv')
data_max_temp = pd.read_csv('./data/data_max_temp.csv')
data_min_temp = pd.read_csv('./data/data_min_temp.csv')
data_palmer_z = pd.read_csv('./data/data_palmer_z.csv')
data_pdsi = pd.read_csv('./data/data_pdsi.csv')
data_phdi = pd.read_csv('./data/data_phdi.csv')
data_pmdi = pd.read_csv('./data/data_pmdi.csv')
data_precipitation = pd.read_csv('./data/data_precipitation.csv')



# Append columns to the data series
data_series['cool_degree_days'] = data_cool_degree['Value']
data_series['cool_degree_days_anomaly'] = data_cool_degree['Anomaly']

data_series['heat_degree_days'] = data_heat_degree['Value']
data_series['heat_degree_days_anomaly'] = data_heat_degree['Anomaly']

data_series['max_temp'] = data_max_temp['Value']
data_series['max_temp_anomaly'] = data_max_temp['Anomaly']

data_series['min_temp'] = data_min_temp['Value']
data_series['min_temp_anomaly'] = data_min_temp['Anomaly']

data_series['palmer_z'] = data_palmer_z['Value']
data_series['palmer_z_anomaly'] = data_palmer_z['Anomaly']

data_series['pdsi'] = data_pdsi['Value']
data_series['pdsi_anomaly'] = data_pdsi['Anomaly']

data_series['phdi'] = data_phdi['Value']
data_series['phdi_anomaly'] = data_phdi['Anomaly']

data_series['pmdi'] = data_pmdi['Value']
data_series['pmdi_anomaly'] = data_pmdi['Anomaly']

data_series['precipitation'] = data_precipitation['Value']
data_series['precipitation_anomaly'] = data_precipitation['Anomaly']




In [72]:
# Interpolate missing nan values based on neighboring values
data_series.interpolate(inplace=True)

In [ ]:
# Convert dataframe to numpy array
avg_temp = data_series['avg_temp'].to_numpy()
data_series.drop(columns=['avg_temp'], inplace=True)
x = data_series.to_numpy()

# Expand first dim
x = np.expand_dims(x, axis=0)
avg_temp = np.expand_dims(avg_temp, axis=0)

# Split the data into training and testing sets
# print(x[0])
# print(avg_temp.shape)

[[ 1.89512e+05  6.21000e+02 -1.96000e+02 ...  4.30000e+00  1.65300e+01
   2.54000e+00]
 [ 1.89601e+05  9.20000e+02  1.03000e+02 ...  1.10000e+00  1.31300e+01
  -8.60000e-01]
 [ 1.89602e+05  7.96000e+02 -2.10000e+01 ...  2.05000e+00  1.74400e+01
   3.45000e+00]
 ...
 [ 2.02010e+05  1.26100e+03  4.44000e+02 ... -4.99000e+00  8.35000e+00
  -5.64000e+00]
 [ 2.02011e+05  1.26100e+03  4.44000e+02 ... -4.99000e+00  8.35000e+00
  -5.64000e+00]
 [ 2.02012e+05  1.26100e+03  4.44000e+02 ... -4.99000e+00  8.35000e+00
  -5.64000e+00]]
(1, 1501)


In [74]:
# Train the GRU model
model = Sequential(
    [
        GRU(20, activation='relu', input_shape=(x.shape[1], x.shape[2])),
        Dense(1)
    ]
)

model.compile(optimizer='adam', loss='mse')
model.fit(x, avg_temp, epochs=100, batch_size=1)

Epoch 1/100


/home/frederik/miniconda3/envs/ml_new_mexico/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 137567856.0000
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 123053424.0000
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 109367208.0000
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 96523088.0000
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 84531592.0000
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 73400080.0000
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 63132148.0000
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 53727384.0000
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 45180824.0000
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 37482256.0000
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 30616314.0000
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 24562164.0000
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 19293220.0000
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 14776804.000